<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#Decision-Tree" data-toc-modified-id="Decision-Tree-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Decision Tree</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LightGBM</a></span></li></ul></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import pandas as pd

import re
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords as nltk_stopwords
import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE=12345

## Подготовка

In [2]:
try:
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')

df = df[['text', 'toxic']]
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


### Лемматизация

In [3]:
df['text'] = df['text'].str.lower()

In [4]:
def prepare_text(text):
    text = re.sub(r'(?:\n|\r)', ' ', text)
    text1 = re.sub(r'[^a-zA-Z]', ' ', text)
    
    return " ".join(text1.split())
    
df['text'] = df['text'].apply(prepare_text)

In [5]:
new_corpus = []
nlp = spacy.load('en_core_web_sm')
for doc in tqdm(nlp.pipe(df['text'], batch_size=70, n_process=-1, disable=['parser', 'ner']), total=len(df['text'])):
    word_list = [tok.lemma_ for tok in doc]
    new_corpus.append(' '.join(word_list))

df['lemm_text'] = new_corpus

100%|█████████████████████████████████████████████████████████████████████████| 159292/159292 [14:22<00:00, 184.75it/s]


In [6]:
df = df.dropna(subset=['lemm_text'])
df

,text,toxic,lemm_text
0,explanation why the edits made under my userna...,0,explanation why the edit make under my usernam...
1,d aww he matches this background colour i m se...,0,d aww he match this background colour I m seem...
2,hey man i m really not trying to edit war it s...,0,hey man I m really not try to edit war it s ju...
3,more i can t make any real suggestions on impr...,0,more I can t make any real suggestion on impro...
4,you sir are my hero any chance you remember wh...,0,you sir be my hero any chance you remember wha...
...,...,...,...
159287,and for the second time of asking when your vi...,0,and for the second time of ask when your view ...
159288,you should be ashamed of yourself that is a ho...,0,you should be ashamed of yourself that be a ho...
159289,spitzer umm theres no actual article for prost...,0,spitzer umm there s no actual article for pros...
159290,and it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


## Обучение

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['lemm_text'], df['toxic'], test_size=.25, random_state=RANDOM_STATE)

In [8]:
vectorizer_tf = TfidfVectorizer(stop_words= set(nltk_stopwords.words('english')))
X_train_tf = vectorizer_tf.fit_transform(X_train)
X_test_tf = vectorizer_tf.transform(X_test)    

### Logistic Regression

Далее ячейки закоментил, т.к. расчет занимает много времени.

In [9]:
# log_reg_model = LogisticRegression(random_state=RANDOM_STATE)

# parameters = [
#     {'C' : [10, 20, 40, 80],
#     'max_iter' : [100, 1000, 2500, 5000]
#     } ]

# log_reg_grid = RandomizedSearchCV(log_reg_model, parameters, scoring='f1').fit(X_train_tf, y_train)
# log_reg_f1 = print(log_reg_grid.best_score_)
# print("Лучшая модель")
# print(log_reg_grid.best_estimator_)

In [10]:
model_lr = LogisticRegression(C=20, max_iter=2500, random_state=RANDOM_STATE)
model_lr.fit(X_train_tf, y_train)

lr_f1 = f1_score(y_train, model_lr.predict(X_train_tf))

### Random Forest

In [11]:
# model_rf = RandomForestClassifier(random_state=RANDOM_STATE)

# parameters = {'n_estimators': [8,16,32,64,128],
#               'max_depth': range(2,22, 4)}

# rf_grid = RandomizedSearchCV(model_rf, parameters, scoring='f1').fit(X_train_tf, y_train)
# rf_rmse = print(rf_grid.best_score_)

# print("Лучшая модель")
# print(rf_grid.best_estimator_)

In [12]:
model_rf = RandomForestClassifier(max_depth=14, n_estimators=32, random_state=RANDOM_STATE)
model_rf.fit(X_train_tf, y_train)

rf_f1 = f1_score(y_train, model_rf.predict(X_train_tf))

### Decision Tree

In [13]:
# model_dt = DecisionTreeClassifier(random_state=RANDOM_STATE)

# parameters = {'max_depth': range(10, 110, 10)}

# dt_grid = RandomizedSearchCV(model_dt, parameters, scoring='f1').fit(X_train_tf, y_train)
# dt_rmse = print(dt_grid.best_score_)

# print("Лучшая модель")
# print(dt_grid.best_estimator_)

In [14]:
model_dt = DecisionTreeClassifier(max_depth = 100, random_state=RANDOM_STATE)
model_dt.fit(X_train_tf, y_train)

dt_f1 = f1_score(y_train, model_dt.predict(X_train_tf))

### LightGBM

In [15]:
# model_lgbm = LGBMClassifier(random_state=RANDOM_STATE)

# parameters = {'max_depth': range (2, 10, 2),
#               'learning_rate': [0.1, 0.5, 0.8]}

# lgbm_grid = RandomizedSearchCV(model_lgbm, parameters, scoring='f1').fit(X_train_tf, y_train)
# lgbm_f1 = print(lgbm_grid.best_score_)
# print("Лучшая модель")
# print(lgbm_grid.best_estimator_)

In [16]:
model_lgbm = LGBMClassifier(learning_rate=0.5, max_depth=8, random_state=RANDOM_STATE)
model_lgbm.fit(X_train_tf, y_train)

lgbm_f1 = f1_score(y_train, model_lgbm.predict(X_train_tf))

## Анализ моделей

In [17]:
result = {}
for i, k in zip(['Logistic Regression', 'Random Forest', 'Decision Tree' ,'LightGBM'], ['lr', 'rf', 'dt','lgbm']):
    result[i] = [vars()[k + '_f1']]

result_table = pd.DataFrame(result.values(), index = result.keys(), columns = ['F1']  )
result_table.style.highlight_max(subset = ['F1'], color = 'lightgreen', axis = 0)

,F1
Logistic Regression,0.941975
Random Forest,0.000165
Decision Tree,0.884299
LightGBM,0.865836


## Тестирование лучшей модели

In [18]:
f1_final = f1_score(y_test, model_lr.predict(X_test_tf))
print(f'F1 лучшей модели на тестовой выборке: {f1_final:.2f}')

F1 лучшей модели на тестовой выборке: 0.79


## Выводы

1. Была произведена загрузка данных и их подготовка.
2. Сформированы обучающая и тестовая выборки.
3. Были обучены 4 модели, на которых были подобраны оптимальные гиперпараметры. Наилучшая метрика F1 у модели LogisticRegression: 0.94.
5. Данная модель была протестирована на тестовой выборке и дала результат 0.79, что выше требуемого значения.